<a href="https://colab.research.google.com/github/Girraffeteeth/WanderHub/blob/main/WanderHub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
import random
import math
from tabulate import tabulate


In [ ]:
# Mount your Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
hotels = pd.read_csv('/content/gdrive/MyDrive/tourism_dataset_south_india.csv')

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
import random
import math

# Mount Google Drive
drive.mount('/content/gdrive')

# User profiling
user_profiles = {}

def create_profile():
    name = input("Enter your name: ")
    preferences = {
        'destination': input("Enter your preferred destination: "),
        'budget': float(input("Enter your budget: ")),
        'rating': float(input("Enter your preferred rating: ")),
        'amenities': input("Enter your preferred amenities (separate by comma): ").split(','),
        'sun_exposure_weight': float(input("Enter your Sun Exposure Weight (0.0 to 1.0): ")),
        'fitness_weight': float(input("Enter your Fitness Weight (0.0 to 1.0): ")),
        'sightseeing_weight': float(input("Enter your Sightseeing Weight (0.0 to 1.0): ")),
        'nightlife_weight': float(input("Enter your Nightlife Weight (0.0 to 1.0): ")),
        'health_weight': float(input("Enter your Health Weight (negative impact): ")),
    }
    user_profiles[name] = preferences
    print("User profile created:")
    print(user_profiles)

# Define a function to get hotel recommendations
def get_recommendations(destination, budget, rating, amenities):
    # Filter hotels based on destination, budget, and rating
    filtered_hotels = hotels[(hotels['destination'] == destination) & (hotels['price'] <= budget) & (hotels['rating'] >= rating)]

    # Find hotels with similar amenities
    if len(amenities) >= 3:
        similar_hotels = filtered_hotels[
            filtered_hotels['amenities'].str.contains(amenities[0]) |
            filtered_hotels['amenities'].str.contains(amenities[1]) |
            filtered_hotels['amenities'].str.contains(amenities[2])
        ]
    elif len(amenities) == 2:
        similar_hotels = filtered_hotels[
            filtered_hotels['amenities'].str.contains(amenities[0]) |
            filtered_hotels['amenities'].str.contains(amenities[1])
        ]
    elif len(amenities) == 1:
        similar_hotels = filtered_hotels[
            filtered_hotels['amenities'].str.contains(amenities[0])
        ]
    else:
        similar_hotels = filtered_hotels

    # Return hotel details
    return similar_hotels[['hotel_name', 'price', 'rating', 'amenities']]

class UserPreferences:
    def __init__(self, sun_exposure_weight, fitness_weight, sightseeing_weight, nightlife_weight, health_weight):
        self.sun_exposure_weight = sun_exposure_weight
        self.fitness_weight = fitness_weight
        self.sightseeing_weight = sightseeing_weight
        self.nightlife_weight = nightlife_weight
        self.health_weight = health_weight

    def calculate_utility(self, itinerary):
        return (self.sun_exposure_weight * itinerary.sun_exposure +
                self.fitness_weight * itinerary.fitness +
                self.sightseeing_weight * itinerary.sightseeing +
                self.nightlife_weight * itinerary.nightlife -
                self.health_weight * itinerary.health_risks)

class Itinerary:
    def __init__(self, sun_exposure, fitness, sightseeing, nightlife, health_risks):
        self.sun_exposure = sun_exposure
        self.fitness = fitness
        self.sightseeing = sightseeing
        self.nightlife = nightlife
        self.health_risks = health_risks

class IntelligentTouristAgent:
    def __init__(self):
        self.random = random.Random()

    def generate_random_itinerary(self):
        return Itinerary(self.random.random(), self.random.random(), self.random.random(), self.random.random(), self.random.random())

    def optimize_itinerary(self, preferences, population_size, generations, temperature, cooling_rate):
        population = [self.generate_random_itinerary() for _ in range(population_size)]

        for _ in range(generations):
            population.sort(key=lambda itinerary: preferences.calculate_utility(itinerary), reverse=True)
            population = population[:population_size // 2]
            new_population = population.copy()
            while len(new_population) < population_size:
                parent1 = random.choice(population)
                parent2 = random.choice(population)
                new_population.append(self.crossover_itineraries(parent1, parent2))

            for itinerary in new_population:
                neighbor = self.generate_random_itinerary()
                neighbor_utility = preferences.calculate_utility(neighbor)
                current_utility = preferences.calculate_utility(itinerary)
                if self.acceptance_probability(current_utility, neighbor_utility, temperature) > random.random():
                    itinerary.sun_exposure = neighbor.sun_exposure
                    itinerary.fitness = neighbor.fitness
                    itinerary.sightseeing = neighbor.sightseeing
                    itinerary.nightlife = neighbor.nightlife
                    itinerary.health_risks = neighbor.health_risks

            temperature *= cooling_rate
            population = new_population

        return population[0]

    def crossover_itineraries(self, it1, it2):
        return Itinerary((it1.sun_exposure + it2.sun_exposure) / 2,
                         (it1.fitness + it2.fitness) / 2,
                         (it1.sightseeing + it2.sightseeing) / 2,
                         (it1.nightlife + it2.nightlife) / 2,
                         (it1.health_risks + it2.health_risks) / 2)

    def acceptance_probability(self, current_utility, neighbor_utility, temperature):
        if neighbor_utility > current_utility:
            return 1.0
        return math.exp((neighbor_utility - current_utility) / temperature)

def get_personalized_recommendations(name):
    if name in user_profiles:
        preferences = user_profiles[name]
        recommendations = get_recommendations(preferences['destination'], preferences['budget'], preferences['rating'], preferences['amenities'])
        optimized_itinerary = IntelligentTouristAgent().optimize_itinerary(UserPreferences(preferences['sun_exposure_weight'], preferences['fitness_weight'], preferences['sightseeing_weight'], preferences['nightlife_weight'], preferences['health_weight']), population_size=100, generations=100, temperature=1.0, cooling_rate=0.1)
        print("Optimized Itinerary:")
        print(tabulate([['Sun Exposure', optimized_itinerary.sun_exposure],
                        ['Fitness', optimized_itinerary.fitness],
                        ['Sightseeing', optimized_itinerary.sightseeing],
                        ['Nightlife', optimized_itinerary.nightlife],
                        ['Health Risks', optimized_itinerary.health_risks]],
                      headers=['Activity', 'Score']))
        return {
            'profile': preferences,
            'recommendations': recommendations,
            'optimized_itinerary': optimized_itinerary
        }
    else:
        return None

# Main program
while True:
    print("1. Create profile")
    print("2. Get personalized recommendations")
    print("3. Exit")
    choice = input("Enter your choice: ")
    if choice == '1':
        create_profile()
    elif choice == '2':
        name = input("Enter your name: ")
        recommendations = get_personalized_recommendations(name)
        if recommendations is not None:
            print("Recommendations for", name)
            print("Based on your profile:")
            print("Name:", name)
            print("Profile:", recommendations['profile'])
            print("Recommendations:")
            print(recommendations['recommendations'])
        else:
            print("Profile not found")
    elif choice == '3':
        print("Exiting program. Goodbye!")
        break
    else:
        print("Invalid choice. Please try again.")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
1. Create profile
2. Get personalized recommendations
3. Exit
Enter your choice: 1
Enter your name: Meghna
Enter your preferred destination: Bengaluru
Enter your budget: 600
Enter your preferred rating: 4
Enter your preferred amenities (separate by comma): free wifi
Enter your Sun Exposure Weight (0.0 to 1.0): 0.5
Enter your Fitness Weight (0.0 to 1.0): 0.7
Enter your Sightseeing Weight (0.0 to 1.0): 0.8
Enter your Nightlife Weight (0.0 to 1.0): 0.3
Enter your Health Weight (negative impact): -0.3
User profile created:
{'Meghna': {'destination': 'Bengaluru', 'budget': 600.0, 'rating': 4.0, 'amenities': ['free wifi'], 'sun_exposure_weight': 0.5, 'fitness_weight': 0.7, 'sightseeing_weight': 0.8, 'nightlife_weight': 0.3, 'health_weight': -0.3}}
1. Create profile
2. Get personalized recommendations
3. Exit
Enter your choice: 2
Enter your name: Meghna
Optimized 